In [1]:
!pip install textattack==0.3.5

  Using cached textattack-0.3.5-py3-none-any.whl.metadata (37 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached flair-0.15.1-py3-none-any.whl.metadata (12 kB)
  Using cached language_tool_python-2.9.3-py3-none-any.whl.metadata (54 kB)
  Using cached lemminflect-0.2.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached lru_dict-1.3.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached datasets-1.15.0-py3-none-any.whl.metadata (19 kB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached word2number-1.1-py3-none-any.whl
  Using cached num2words-0.5.14-py3-none-any.whl.metadata (13 kB)
  Using cached pinyin-0.4.0-py3-none-any.whl
  Using cached OpenHowNet-2.0-py3-none-any.whl.metadata (821 bytes)
  Using cached pycld2-0.42-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached click-8.0.4-py3-none-any.whl

In [2]:
!pip install transformers torch sentencepiece

In [3]:
# 3. JSON 데이터 업로드
from google.colab import files
import json

uploaded = files.upload()

with open("knowledge.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

Saving knowledge.json to knowledge (1).json


In [4]:
# 라이브러리 import
import random
import json
import requests
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from google.colab import files

# 1. Papago API 설정
client_id = '64yuz72rjr'
client_secret = 'mpxNeA4kw4gIEe2TgfRDfnWSzeWBJ1Sz0FfLkUDW'

def papago_translate(text, source_lang, target_lang):
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    data = {
        "source": source_lang,
        "target": target_lang,
        "text": text
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['message']['result']['translatedText']
    else:
        print(f"번역 오류 발생: {response.status_code}, 응답: {response.text}")
        return None

# 2. HuggingFace T5 Paraphraser 로딩
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_paraphrases(text, num_return_sequences=3):
    input_text = f"paraphrase: {text} </s>"
    outputs = paraphraser(
        input_text,
        max_length=100,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=100,
        top_p=0.92,
        temperature=0.8
    )
    return [output['generated_text'] for output in outputs]

# 3. Synonym 치환
synonym_dict = {
    # 명사 치환
    "수도": ["중심 도시", "중앙 도시"],
    "수학자": ["수학 연구자", "수학 전문가"],
    "장군": ["지휘관", "군사 지도자"],
    "정책": ["방침", "계획"],
    "현상": ["이론", "상태", "반응"],
    "기호": ["표기", "상징"],
    "의미": ["뜻", "정의"],
    "제도": ["시스템", "체계"],
    "약자": ["줄임말", "축약어"],

    # 동사 치환
    "표기하다": ["적다", "기록하다"],
    "뜻하다": ["의미하다", "가리키다"],
    "전달하다": ["전송하다", "보내다"],
    "지칭하다": ["일컫다", "부르다"],
    "이루어지다": ["구성되다", "형성되다"],

    # 형용사/부사 치환
    "유명한": ["잘 알려진", "이름난"],
    "대표적인": ["주요한", "전형적인"],
    "가장 큰": ["최대의", "제일 큰"],
    "빠르게": ["신속하게", "재빠르게"],
    "간단한": ["단순한", "쉽게 이해되는"],
}

def synonym_replace(korean_text):
    words = korean_text.split()
    new_words = []
    for word in words:
        replaced = False
        for key, synonyms in synonym_dict.items():
            if key in word:
                if random.random() < 0.5:
                    word = word.replace(key, random.choice(synonyms))
                    replaced = True
                    break
        new_words.append(word)
    return ' '.join(new_words)

# 4. Style Transfer
def style_transfer(text, apply_prob=0.6):
    if random.random() > apply_prob:
        return text

    endings = {
      # 종결 표현
      "무엇일까요": ["뭐야", "뭔데"],
      "무엇인가요": ["뭐야", "뭔지 알아?"],
      "어디일까요": ["어디야", "어디에 있어?"],
      "얼마일까요": ["얼마야", "얼만데"],
      "전부 말하시오": ["전부 알려줘", "다 말해봐"],
      "설명하시오": ["설명해줘", "알려줄래"],

      # 표현 전환
      "의 이름은": ["는 이름이야", "라고 불려"],
      "로 알려진": ["라고 하는", "로 불리는"],
      "라고 불리는": ["라고 하는", "라고 알려진"],

      # 높임 표현 → 평어체
      "입니다": ["야", "이지"],
      "입니다.": ["야.", "이지."],
      "일까요?": ["야?", "지?"],
      "인가요?": ["야?", "지?"]
    }

    for formal, casual_list in endings.items():
        if formal in text:
            text = text.replace(formal, random.choice(casual_list))
    return text

# 5. 전체 프로세스 함수
def full_back_translate_with_paraphrase_style(korean_text, num_augments=5):
    en_text = papago_translate(korean_text, source_lang='ko', target_lang='en')
    time.sleep(0.2)

    paraphrases = generate_paraphrases(en_text, num_return_sequences=num_augments)
    time.sleep(0.2)

    results = []
    for p in paraphrases:
        ko_text = papago_translate(p, source_lang='en', target_lang='ko')
        time.sleep(0.2)

        ko_text_synonym = synonym_replace(ko_text)
        final_text = style_transfer(ko_text_synonym)

        results.append(final_text)

    return results

# 7. 증강하기
augmented_data = []

for item in raw_data:
    question = item['Question']
    answer = item['Answer']

    # 질문을 여러 개 증강
    augmented_questions = full_back_translate_with_paraphrase_style(question, num_augments=5)

    for aug_q in augmented_questions:
        augmented_data.append({
            "Question": aug_q,
            "Answer": answer
        })

# 8. 증강된 데이터 저장
output_filename = "augmented_knowledge.json"

with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)

# 9. Colab에서 바로 다운로드
files.download(output_filename)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>